# ETL: Camada Silver $\rightarrow$ Camada Gold (Data Warehouse)

**Projeto:** Social Media User Analysis  
**Origem:** Schema `silver` (PostgreSQL - Tabela tratada)  
**Destino:** Schema `dw` (PostgreSQL - Modelagem Dimensional Star Schema)

---

## Objetivo
Transformar os dados limpos da camada Silver em um modelo dimensional otimizado para consultas analíticas (OLAP). O modelo escolhido é o Star Schema, composto por 2 Tabelas Fato e 4 Tabelas Dimensão.

## Arquitetura do Data Warehouse

### 1. Tabelas Fato (Métricas)
* **`FT_ADS_PRF` (Ads Performance):** Foca na eficiência da publicidade (Views, Cliques, CTR).
* **`FT_ENG_APP` (Engajamento App):** Foca no comportamento do usuário (Likes, Comentários, Tempo de Tela).

### 2. Tabelas Dimensão (Contexto)
* **`DIM_USR` (Usuário):** Dados demográficos (Idade, Gênero, País).
* **`DIM_ETL_VDA` (Estilo de Vida):** Saúde e hábitos (Sono, Exercício, Felicidade).
* **`DIM_CNT` (Conta):** Configurações do app (Premium, Privacidade, Data de Criação).
* **`DIM_ITR` (Interesse):** Preferências de conteúdo (Tech, Fashion, etc.).


In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
import numpy as np
import warnings

warnings.filterwarnings('ignore', category=UserWarning, message='.*SQLAlchemy.*')
     
# --- CONFIGURAÇÕES DE CONEXÃO ---
DB_HOST = "localhost"
DB_NAME = "instagram_usage"
DB_USER = "sbd2"
DB_PASS = "sbd2123"  

# Schemas
SILVER_SCHEMA = "silver"
SILVER_TABLE = "user"
GOLD_SCHEMA = "dw"

## Etapa 1: Extração
Nesta etapa, conectamos ao banco de dados PostgreSQL e lemos todos os dados da tabela `silver.social_media_silver`. 



In [2]:
try:
    conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASS)
    query = f"SELECT * FROM {SILVER_SCHEMA}.{SILVER_TABLE}"
    
    # Leitura para DataFrame
    df_silver = pd.read_sql(query, conn)
    print(f"Dados carregados da camada Silver com sucesso!")
    print(f" Registros: {df_silver.shape[0]} | Colunas: {df_silver.shape[1]}")
    
    conn.close()
except Exception as e:
    print(f"Erro na extração: {e}")

Dados carregados da camada Silver com sucesso!
 Registros: 1506286 | Colunas: 58


##  Etapa 2: Transformação

Aqui ocorre a modelagem dimensional. O processo é dividido em sub-etapas:

1.  **Mapeamento de Colunas:** Definição dos dicionários `De -> Para` para aplicar os nomes mnemônicos (ex: `age` vira `AGE_USR`).
2.  **Criação das Dimensões:**
    * Selecionamos as colunas de atributos únicos.
    * Aplicamos `drop_duplicates()` para garantir que cada perfil exista apenas uma vez.
    * Geramos as **Surrogate Keys (SRK)** sequenciais (1, 2, 3...) para identificar cada linha.
3.  **Criação das Fatos:**
    * Fazemos o `merge` (Join) da tabela base com as Dimensões criadas para recuperar os IDs (`SRK`) corretos.
    * Separamos as métricas de Ads e Engajamento em dois DataFrames distintos.
    * Geramos uma `SRK` própria para cada tabela fato (`SRK_ADS` e `SRK_ENG`), conforme requisito do projeto.

In [3]:
# --- 2.1 Dicionários de Mapeamento  ---

# Mapeamento: Usuário
map_usuario = {
    'sk_usuario': 'SRK_USR', 'user_id': 'SRK_USR', 
    'age': 'AGE_USR', 'gender': 'GEN_USR', 'country': 'CTR_USR',
    'urban_rural': 'URB_RRL', 'education_level': 'EDU_LVL',
    'employment_status': 'EMP_STS', 'income_level': 'INC_LVL',
    'relationship_status': 'REL_STS', 'has_children': 'HAS_CHD'
}

# Mapeamento: Estilo de Vida
map_estilovida = {
    'sk_estilovida': 'SRK_ETL_VDA',
    'exercise_hours_per_week': 'EXE_HRS_WEK', 'sleep_hours_per_night': 'SLP_HRS_NGT',
    'diet_quality': 'DIT_QLT', 'body_mass_index': 'BDY_MAS_IDX',
    'blood_pressure_systolic': 'BLD_PRS_SYS', 'blood_pressure_diastolic': 'BLD_PRS_DIA',
    'daily_steps_count': 'DLY_STP_CNT', 'perceived_stress_score': 'STR_SCR',
    'self_reported_happiness': 'HPN_SCR', 'smoking': 'SMK_FLG',
    'alcohol_frequency': 'ALC_FRQ', 'weekly_work_hours': 'WRK_HRS_WEK',
    'hobbies_count': 'HOB_CNT', 'social_events_per_month': 'SOC_EVT_MTH',
    'books_read_per_year': 'BKS_RED_YAR', 'volunteer_hours_per_month': 'VOL_HRS_MTH',
    'travel_frequency_per_year': 'TRV_FRQ_YAR'
}

# Mapeamento: Conta
map_conta = {
    'sk_conta': 'SRK_CNT',
    'app_name': 'APP_NME', 'account_creation_year': 'ACC_CRT_YAR',
    'last_login_date': 'LST_LOG_DTE', 'subscription_status': 'SUB_STS',
    'uses_premium_features': 'USE_PRM_FTR', 'privacy_setting_level': 'PRV_LVL',
    'two_factor_auth_enabled': 'TWO_FAC_AUT', 'biometric_login_used': 'BIO_LOG_USE'
}

# Mapeamento: Interesse
map_interesse = {
    'sk_interesse': 'SRK_ITR',
    'content_type_preference': 'CNT_TYP_PRF',
    'preferred_content_theme': 'CNT_TME_PRF'
}

# Mapeamento: Chaves Estrangeiras nas Fatos
map_fatos_fk = {
    'sk_usuario': 'SRK_USR', 'sk_estilovida': 'SRK_ETL_VDA',
    'sk_conta': 'SRK_CNT', 'sk_interesse': 'SRK_ITR'
}

# Métricas das Fatos
map_ads = {'ads_viewed_per_day': 'ADS_VIW_DIA', 'ads_clicked_per_day': 'ADS_CLK_DIA'}

map_eng = {
    'daily_active_minutes_instagram': 'DLY_ACT_MIN', 'sessions_per_day': 'SES_DIA',
    'average_session_length_minutes': 'AVG_SES_MIN', 'user_engagement_score': 'ENG_SCR',
    'likes_given_per_day': 'LIK_GVN_DIA', 'comments_written_per_day': 'COM_WRT_DIA',
    'posts_created_per_week': 'PST_CRT_WEK', 'dms_sent_per_week': 'DMS_SNT_WEK',
    'dms_received_per_week': 'DMS_RCV_WEK', 'reels_watched_per_day': 'RLS_WCH_DIA',
    'stories_viewed_per_day': 'STR_VIW_DIA', 'time_on_feed_per_day': 'TIM_FED_DIA',
    'time_on_explore_per_day': 'TIM_EXP_DIA', 'time_on_reels_per_day': 'TIM_RLS_DIA',
    'time_on_messages_per_day': 'TIM_MSG_DIA', 'followers_count': 'FOL_CNT',
    'following_count': 'FLW_CNT', 'linked_accounts_count': 'LNK_ACC_CNT',
    'notification_response_rate': 'NTF_RSP_RAT'
}

In [4]:
# --- 2.2 Geração das Tabelas Dimensão ---

# 1. DIM_USR (Dimensão Usuário)
cols_attrs_usuario = ['age', 'gender', 'country', 'urban_rural', 'income_level', 
                      'employment_status', 'education_level', 'relationship_status', 'has_children']

dim_usuario = df_silver[['user_id'] + cols_attrs_usuario].drop_duplicates().reset_index(drop=True)
dim_usuario['sk_usuario'] = dim_usuario['user_id']
dim_usuario = dim_usuario[['sk_usuario'] + cols_attrs_usuario]
dim_usuario.rename(columns=map_usuario, inplace=True)

# 2. DIM_ETL_VDA (Dimensão Estilo de Vida)
# Estratégia: Criar ID sequencial novo
cols_estilovida = list(map_estilovida.keys())
cols_estilovida.remove('sk_estilovida')
dim_estilovida = df_silver[cols_estilovida].drop_duplicates().reset_index(drop=True)
dim_estilovida['sk_estilovida'] = dim_estilovida.index + 1
dim_estilovida = dim_estilovida[['sk_estilovida'] + cols_estilovida]
dim_estilovida.rename(columns=map_estilovida, inplace=True)

# 3. DIM_CNT (Dimensão Conta)
cols_conta = list(map_conta.keys())
cols_conta.remove('sk_conta')
dim_conta = df_silver[cols_conta].drop_duplicates().reset_index(drop=True)
dim_conta['sk_conta'] = dim_conta.index + 1
dim_conta = dim_conta[['sk_conta'] + cols_conta]
dim_conta.rename(columns=map_conta, inplace=True)

# 4. DIM_ITR (Dimensão Interesse)
cols_interesse = list(map_interesse.keys())
cols_interesse.remove('sk_interesse')
dim_interesse = df_silver[cols_interesse].drop_duplicates().reset_index(drop=True)
dim_interesse['sk_interesse'] = dim_interesse.index + 1
dim_interesse = dim_interesse[['sk_interesse'] + cols_interesse]
dim_interesse.rename(columns=map_interesse, inplace=True)

print("Dimensões geradas com sucesso.")


Dimensões geradas com sucesso.


In [5]:
# --- 2.3 Geração das Tabelas Fato ---

# Preparação: Unir a base original com as Dimensões criadas para pegar os IDs (FKs)
fato_base = df_silver.copy()
fato_base['sk_usuario'] = fato_base['user_id'] # SRK_USR é igual ao ID

# Merge (Left Join) para buscar as chaves SRK_ETL_VDA, SRK_CNT, SRK_ITR
# Utilizamos as colunas de atributos como chave de busca
fato_base = fato_base.merge(dim_estilovida.rename(columns={v:k for k,v in map_estilovida.items()}), 
                            on=list(map_estilovida.keys())[1:], how='left')
fato_base = fato_base.merge(dim_conta.rename(columns={v:k for k,v in map_conta.items()}), 
                            on=list(map_conta.keys())[1:], how='left')
fato_base = fato_base.merge(dim_interesse.rename(columns={v:k for k,v in map_interesse.items()}), 
                            on=list(map_interesse.keys())[1:], how='left')

cols_fk = ['sk_usuario', 'sk_estilovida', 'sk_conta', 'sk_interesse']

# --- FATO 1: FT_ADS_PRF (Performance de Ads) ---
cols_ads = list(map_ads.keys())
fato_ads = fato_base[cols_fk + cols_ads].copy()
fato_ads.rename(columns={**map_fatos_fk, **map_ads}, inplace=True)

# Gerando chave primária própria (SRK_ADS)
fato_ads.reset_index(drop=True, inplace=True)
fato_ads['SRK_ADS'] = fato_ads.index + 1
cols_order_ads = ['SRK_ADS'] + [c for c in fato_ads.columns if c != 'SRK_ADS']
fato_ads = fato_ads[cols_order_ads]

# --- FATO 2: FT_ENG_APP (Engajamento do App) ---
cols_eng = list(map_eng.keys())
fato_eng = fato_base[cols_fk + cols_eng].copy()
fato_eng.rename(columns={**map_fatos_fk, **map_eng}, inplace=True)

# Gerando chave primária própria (SRK_ENG)
fato_eng.reset_index(drop=True, inplace=True)
fato_eng['SRK_ENG'] = fato_eng.index + 1
cols_order_eng = ['SRK_ENG'] + [c for c in fato_eng.columns if c != 'SRK_ENG']
fato_eng = fato_eng[cols_order_eng]

print(f"Fatos gerados com sucesso.")

Fatos geradas com sucesso.


##  Etapa 3: Carga

A etapa final persiste os DataFrames transformados no banco de dados.

**Estratégia de Carga:** `Full Refresh` (Truncate & Insert)
1.  Limpamos as tabelas existentes (TRUNCATE CASCADE) para evitar duplicação.
2.  Inserimos os dados em lote (Batch Insert) para alta performance.

**Ordem de Inserção (Crítica para Integridade Referencial):**
1.  **Dimensões** (Pois as fatos dependem delas).
2.  **Fatos** (Que referenciam as dimensões).

In [6]:
def insert_data(cur, df, table_name):
    """
    Função auxiliar para inserir dados em lote no PostgreSQL.
    """
    if df.empty: 
        return
    
    # Prepara a query SQL dinâmica
    cols = df.columns.tolist()
    cols_str = ", ".join(cols)
    placeholders = ", ".join(["%s"] * len(cols))
    sql = f"INSERT INTO {GOLD_SCHEMA}.{table_name} ({cols_str}) VALUES ({placeholders})"
    
    # Conversão de dados (DataFrame -> Lista de Tuplas)
    # .replace({np.nan: None}) garante que NaNs virem NULL no SQL
    data = [tuple(x) for x in df.replace({np.nan: None}).to_numpy().tolist()]
    
    try:
        execute_batch(cur, sql, data)
        print(f"   -> Inseridos {len(data)} registros em {table_name}")
    except Exception as e:
        print(f"   -> Erro ao inserir em {table_name}: {e}")
        raise e

try:
    conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASS)
    cur = conn.cursor()
    
    print("Iniciando Carga no Data Warehouse...")
    
    # 1. Limpeza (Ordem: Fatos primeiro, depois Dimensões devido a FKs)
    tables_to_clear = ['FT_ADS_PRF', 'FT_ENG_APP', 'DIM_USR', 'DIM_ETL_VDA', 'DIM_CNT', 'DIM_ITR']
    for t in tables_to_clear:
        cur.execute(f"TRUNCATE TABLE {GOLD_SCHEMA}.{t} CASCADE;")
    print(" Tabelas antigas limpas.")
    
    # 2. Inserção das Dimensões
    insert_data(cur, dim_usuario, 'DIM_USR')
    insert_data(cur, dim_estilovida, 'DIM_ETL_VDA')
    insert_data(cur, dim_conta, 'DIM_CNT')
    insert_data(cur, dim_interesse, 'DIM_ITR')
    
    # 3. Inserção das Fatos
    insert_data(cur, fato_ads, 'FT_ADS_PRF')
    insert_data(cur, fato_eng, 'FT_ENG_APP')
    
    conn.commit()
    print("\nSUCESSO! Carga ETL concluída. O Data Warehouse está atualizado.")

except Exception as e:
    conn.rollback()
    print(f"\nFALHA CRÍTICA na carga: {e}")
finally:
    if conn:
        conn.close()

🔄 Iniciando Carga no Data Warehouse...
   ✅ Tabelas antigas limpas.
   -> Inseridos 1506286 registros em DIM_USR
   -> Inseridos 1506286 registros em DIM_ETL_VDA
   -> Inseridos 315973 registros em DIM_CNT
   -> Inseridos 48 registros em DIM_ITR
   -> Inseridos 1506286 registros em FT_ADS_PRF
   -> Inseridos 1506286 registros em FT_ENG_APP

SUCESSO! Carga ETL concluída. O Data Warehouse está atualizado.
